In [1]:
%pip install yfinance
%pip install pandas
%pip install numpy

In [2]:
import yfinance as yf
import numpy as np
import pandas as pd

In [7]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from datetime import datetime


ticker = "AAPL"
start_date = "2023-01-01"
end_date = datetime.today().strftime("%Y-%m-%d")


try:
    data = yf.download(
        tickers=ticker,
        start=start_date,
        end=end_date,
        progress=True  # 显示下载进度条
    )

    # 确保数据不为空
    if data.empty:
        raise ValueError("没有获取到数据，请检查股票代码和日期范围")

except Exception as e:
    print(f"下载数据时出错: {e}")
    exit()


data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
data['SMA_50'] = data['Close'].rolling(window=50, min_periods=1).mean()

print(data)


[*********************100%***********************]  1 of 1 completed

Price             Open        High         Low       Close     Volume  \
Ticker            AAPL        AAPL        AAPL        AAPL       AAPL   
Date                                                                    
2023-01-03  128.613985  129.226052  122.582119  123.470612  112117500   
2023-01-04  125.267347  127.014716  123.480495  124.744125   89113600   
2023-01-05  125.504260  126.136075  123.164572  123.421242   80962700   
2023-01-06  124.398597  128.623856  123.292916  127.962425   87754700   
2023-01-09  128.801572  131.703978  128.228987  128.485657   70790800   
...                ...         ...         ...         ...        ...   
2025-05-23  193.669998  197.699997  193.460007  195.270004   78432900   
2025-05-27  198.300003  200.740005  197.429993  200.210007   56288500   
2025-05-28  200.589996  202.729996  199.899994  200.419998   45339700   
2025-05-29  203.580002  203.809998  198.509995  199.949997   51477900   
2025-05-30  199.369995  201.960007  196.779999  200

## 成本计算

In [5]:
def calculate_trade_cost(order):
    """计算交易总成本"""
    # 佣金
    commission = max(order.shares * 0.005, 1)  # $0.005/股 最低$1

    # 滑点模型（基于流动性）
    liquidity_factor = 0.0005 * (order.shares / 10000)**0.5
    slippage = order.price * liquidity_factor

    # 市场冲击（大额订单）
    impact = 0.0002 * order.shares if order.shares > 10000 else 0

    total_cost = commission + slippage + impact
    return total_cost